In [1]:
import pandas as pd
df = pd.read_csv('/content/100_Unique_QA_Dataset.csv')
df.head()

,question,answer
0,What is the capital of France?,Paris
1,What is the capital of Germany?,Berlin
2,Who wrote 'To Kill a Mockingbird'?,Harper-Lee
3,What is the largest planet in our solar system?,Jupiter
4,What is the boiling point of water in Celsius?,100


In [2]:
# simple tokenize function

def tokenize(text):
  text = text.lower()
  text = text.replace('?','')
  text = text.replace("'","")
  return text.split()


In [3]:
tokenize("What is the capital of France?")

['what', 'is', 'the', 'capital', 'of', 'france']

In [11]:
# vocab
vocab = {'<UNK>':0}

In [6]:
def build_vocab(row):
  tokenized_question = tokenize(row['question'])
  tokenized_answer = tokenize(row['answer'])

  merged_tokens = tokenized_question + tokenized_answer

  for token in merged_tokens:
    if token not in vocab:
      vocab[token] = len(vocab)


In [8]:
# Convert  words to numerical indices

def text_to_indices(text,vocab):

  indexed_text = []

  for token in tokenize(text):
    if token in vocab:
      indexed_text.append(vocab[token])
    else:
      indexed_text.append(vocab['<UNK>'])

  return indexed_text


In [13]:
text_to_indices("What is AI",vocab)

[0, 0, 0]

In [14]:
import torch
from torch.utils.data import Dataset, DataLoader

In [16]:
class QADataset(Dataset):

  def __init__(self,dff,vocab):
    self.df = df
    self.vocab = vocab
  def __len__(self):
    return self.df.shape[0]
  def __getitem__(self,index):
     numerical_question=text_to_indices(self.df.iloc[index]['question'],self.vocab)
     numerical_answer=text_to_indices(self.df.iloc[index]['answer'],self.vocab)

     return torch.tensor(numerical_question),torch.tensor(numerical_answer)


In [17]:
dataset = QADataset(df,vocab)


In [19]:
dataloader = DataLoader(dataset,batch_size=1,shuffle=True)

In [20]:
for question,answer in dataloader:
  print(question,answer)

tensor([[0, 0, 0, 0, 0, 0]]) tensor([[0]])
tensor([[0, 0, 0, 0, 0, 0, 0, 0, 0]]) tensor([[0]])
tensor([[0, 0, 0, 0]]) tensor([[0]])
tensor([[0, 0, 0, 0, 0, 0, 0, 0]]) tensor([[0]])
tensor([[0, 0, 0, 0, 0, 0, 0]]) tensor([[0]])
tensor([[0, 0, 0, 0, 0, 0]]) tensor([[0]])
tensor([[0, 0, 0, 0, 0, 0]]) tensor([[0]])
tensor([[0, 0, 0, 0, 0, 0, 0]]) tensor([[0]])
tensor([[0, 0, 0, 0, 0, 0, 0]]) tensor([[0]])
tensor([[0, 0, 0, 0, 0]]) tensor([[0]])
tensor([[0, 0, 0, 0, 0, 0]]) tensor([[0]])
tensor([[0, 0, 0, 0, 0, 0, 0]]) tensor([[0]])
tensor([[0, 0, 0, 0, 0, 0]]) tensor([[0]])
tensor([[0, 0, 0, 0, 0, 0]]) tensor([[0]])
tensor([[0, 0, 0, 0, 0, 0, 0]]) tensor([[0]])
tensor([[0, 0, 0, 0, 0, 0, 0, 0]]) tensor([[0]])
tensor([[0, 0, 0, 0, 0, 0, 0]]) tensor([[0]])
tensor([[0, 0, 0, 0, 0, 0, 0]]) tensor([[0]])
tensor([[0, 0, 0, 0, 0, 0, 0, 0, 0]]) tensor([[0]])
tensor([[0, 0, 0]]) tensor([[0]])
tensor([[0, 0, 0, 0, 0, 0, 0]]) tensor([[0]])
tensor([[0, 0, 0, 0, 0, 0]]) tensor([[0]])
tensor([[0, 0, 0, 

In [21]:
import torch.nn as nn

In [36]:
class SimpleRNN(nn.Module):

  def __init__(self,vocab_size):
    super().__init__()
    self.embedding =nn.Embedding(vocab_size,embedding_dim=50)
    self.rnn=nn.RNN(50,64)
    self.fc =nn.Linear(64,vocab_size)
  def forward(self,question):
    embedded_question = self.embedding(question)
    hidden,final = self.rnn(embedded_question)
    output = self.fc(final)
    return output



In [37]:
learning_rate = 0.001
epochs = 20

In [38]:
model = SimpleRNN(len(vocab))

In [41]:
 criterion=nn.CrossEntropyLoss()
 optimizer =torch.optim.Adam(model.parameters(),lr=learning_rate)

In [42]:
# training loop

for epoch in range(epochs):
  total_loss = 0
  for question,answer in dataloader:
    optimizer.zero_grad()

    # forward pass
    output=model(question)

    # loss
    loss =criterion(output,answer)

    # gradients

    loss.backward()

    # update
    optimizer.step()

    total_loss = total_loss + loss.item()

  print(f"Epoch {epoch+1}, Loss:{total_loss:4f}")




Epoch 1, Loss:172.273200
Epoch 2, Loss:172.273199
Epoch 3, Loss:172.273199
Epoch 4, Loss:172.273198
Epoch 5, Loss:172.273197
Epoch 6, Loss:172.273199
Epoch 7, Loss:172.273196
Epoch 8, Loss:172.273198
Epoch 9, Loss:172.273195
Epoch 10, Loss:172.273196
Epoch 11, Loss:172.273200
Epoch 12, Loss:172.273195
Epoch 13, Loss:172.273197
Epoch 14, Loss:172.273197
Epoch 15, Loss:172.273194
Epoch 16, Loss:172.273195
Epoch 17, Loss:172.273200
Epoch 18, Loss:172.273196
Epoch 19, Loss:172.273200
Epoch 20, Loss:172.273200


In [43]:
def predict(model, question, threshold=0.5):

  # convert question to numbers
  numerical_question = text_to_indices(question, vocab)

  # tensor
  question_tensor = torch.tensor(numerical_question).unsqueeze(0)

  # send to model
  output = model(question_tensor)

  # convert logits to probs
  probs = torch.nn.functional.softmax(output, dim=1)

  # find index of max prob
  value, index = torch.max(probs, dim=1)

  if value < threshold:
    print("I don't know")

  print(list(vocab.keys())[index])